In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/glade/u/home/linnia/ctsm6_ppe/')
from utils.pyfunctions import *
utils_path = '/glade/u/home/linnia/ctsm6_ppe/utils/'

2025-08-06 11:46:42.341754: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-06 11:46:43.491161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754502403.777930   64611 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754502403.865240   64611 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-06 11:46:44.575157: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
def write_paramset_ds(default_ds, param_ranges_file, paramset_csv, outfile):
    """
    Create and save a NetCDF file of normalized parameter settings for ensemble.

    Parameters:
    -----------
    default_params : dict
        Dictionary of default parameter values (currently unused).
    param_ranges_file : str
        Path to the CSV file with parameter names and inclusion flags.
    paramset_csv : str
        Path to the CSV file containing parameter values for each ensemble member.
    outfile : str
        Output NetCDF file path (should end in .nc).
    """

    # Read parameter names from LHC file
    lhc_key = '/glade/work/linnia/CLM6-PPE/ctsm6_lhc/ctsm6lhc_11262024.txt'
    params_lhc = pd.read_csv(lhc_key).drop(columns='member')

    # Define PFT-specific parameters
    pft_param_names = [
        'kmax', 'psi50', 'jmaxb0', 'slatop', 'lmr_intercept_atkin',
        'medlynslope', 'medlynintercept', 'froot_leaf', 'leafcn',
        'leaf_long', 'KCN', 'dleaf', 'r_mort', 'fsr_pft', 'xl'
    ]

    # Select parameters we are calibrating only
    df = pd.read_csv(param_ranges_file)
    param_names = df[df['include'] == 1]['param'].values

    # Read ensemble values
    paramsets = pd.read_csv(paramset_csv)

    # Initialize 
    pft_ids = [1,2,3,4,5,6,7,8,10,11,12,13,14]
    extra_pfts = [0,9]
    c3_crop_pft = 15
    c3_grass_pft = 13
    
    data_vars = {}
    n_ens = len(paramsets)
    n_pfts = 79

    for param in param_names:
        if param in pft_param_names:
            default_vals = default_ds[param].values
            if len(default_vals) < n_pfts:
                padded_defaults = np.full(n_pfts, 0.5)
                padded_defaults[:len(default_vals)] = default_vals
            else:
                padded_defaults = default_vals
                
            arr = np.tile(padded_defaults, (n_ens, 1))

            # overwrite with the calibrated values
            cols = [f"{param}_{i}" for i in pft_ids]
            arr[:, pft_ids] = paramsets[cols].values

            # Patch extra PFTs
            for p in extra_pfts:
                arr[:, p] = np.tile(default_ds[param].values[p], (n_ens))
            arr[:, c3_crop_pft] = arr[:, c3_grass_pft]

            data_vars[param] = (["ensemble", "pft"], arr)
        else:
            arr = paramsets[param].values
            data_vars[param] = (["ensemble"], arr)

    ds_out = xr.Dataset(
        data_vars,
        coords={
            "ensemble": np.arange(n_ens),
            "pft": np.arange(n_pfts)
        }
    )

    ds_out.to_netcdf(outfile)


### Write paramsets

In [ ]:
default_params = xr.open_dataset(utils_path+'default_params_norm.nc')
param_ranges='/glade/u/home/linnia/ctsm6_ppe/gen_ensembles/gen_paramfiles/ctsm6cal_paramranges_forIcases_08052025.csv'
paramset_csv = './paramsets/caltest_103-107.csv'
outfile = './paramsets/caltest_103-107.nc'

In [8]:
write_paramset_ds(default_params, param_ranges, paramset_csv, outfile)